<a href="https://colab.research.google.com/github/davis9ja/im-srg_tensorflow/blob/master/testing_tensorflow_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import sys
print(tf.__version__)

1.13.1


In [24]:
# --- BUILD HAMILTONIAN -----------
ref = np.asarray([1,1,1,1,0,0,0,0])
holes = np.asarray([0,1,2,3])
particles = np.asarray([4,5,6,7])

# one-body basis is just all possible single particle states
# B1 = tf.constant(holes+particles)#np.append(holes,particles)
B1 = np.append(holes,particles)

# we build two-body basis with all possible pairs of single particle states
B2 = []
ind2B = {}

count = 0
for h in holes:
    for h2 in holes:
        B2.append((h,h2))
        ind2B[(h,h2)] = count
        count += 1

for h in holes:
    for p in particles:
        B2.append((h,p))
        ind2B[(h,p)] = count
        count += 1
        
for p in particles:
    for h in holes:
        B2.append((p,h))
        ind2B[(p,h)] = count
        count += 1
            
for p in particles:
    for p2 in particles:
        B2.append((p,p2)) 
        ind2B[(p,p2)] = count
        count += 1           

# the one-body Hamiltonian is diagonal, with elements given by P-1 or 
# single particle state index divided by 2 (rounding down)
H1B = np.zeros((len(B1), len(B1)))
for i in range(len(B1)):
    H1B[i,i] = np.floor_divide(i,2)

# the two-body Hamiltonian is sparse; only non-zero contributions 
# where pq=rs or pq=sr=-rs
H2B = np.zeros((len(B2), len(B2)))
for i in range(len(B2)):
    for j in range(len(B2)):
        p, q = B2[i]
        r, s = B2[j]
        
        pp = np.floor_divide(p,2)
        qp = np.floor_divide(q,2)
        rp = np.floor_divide(r,2)
        sp = np.floor_divide(s,2)
        
        ps = 1 if p%2==0 else -1
        qs = 1 if q%2==0 else -1
        rs = 1 if r%2==0 else -1
        ss = 1 if s%2==0 else -1
        
        if pp != qp or rp != sp:
            continue
        if ps == qs or rs == ss:
            continue
        if ps == rs and qs == ss:
            H2B[i,j] = -0.25
        if ps == ss and qs == rs:
            H2B[i,j] = 0.25

B1_len = len(B1)
H2B_t = np.zeros((B1_len,B1_len,B1_len,B1_len))
for p in B1:
    for q in B1:
        for r in B1:
            for s in B1:
                H2B_t[p,q,r,s] = H2B[ind2B[(p,q)], ind2B[(r,s)]]
        
# test2 = np.einsum('ijij->ij',H2B_t)
# print(test2)
# for i in holes:
#   for j in holes:
#         print(H2B[ind2B[(i,j)], ind2B[(i,j)]])
#         print(H2B_t[i,j,i,j])
# print(np.shape(list(map(lambda i: H2B_t[i,:,i,:], holes))))
# test = list(map(lambda i,j: H2B_t[i,j,i,j],holes,particles))
# print(np.shape(test))

# covers na - nb
occA = np.zeros_like(H2B_t)
for a in B1:
    for b in B1:
        occA[a,b,a,b] = ref[a] - ref[b]
        
# covers (1-na-nb)
occB = np.zeros_like(H2B_t)
for a in B1:
    for b in B1:
        occB[a,b,a,b] = 1 - ref[a] - ref[b]
        
# covers na*nb + (1-na-nb)*nc
occC = np.zeros((B1_len,B1_len,B1_len,B1_len,B1_len,B1_len))
for a in B1:
    for b in B1:
        for c in B1:
            occC[a,b,c,a,b,c] = ref[a]*ref[b] + (1-ref[a]-ref[b])*ref[c]



ref_t = tf.convert_to_tensor(ref, dtype=tf.int64)

holes = tf.convert_to_tensor(holes, dtype=tf.int64)
particles = tf.convert_to_tensor(particles, dtype=tf.int64)

H1B_t = tf.convert_to_tensor(H1B, dtype=tf.float64)
H2B_t = tf.convert_to_tensor(H2B_t, dtype=tf.float64)

occA_t = tf.convert_to_tensor(occA, dtype=tf.float64)
occB_t = tf.convert_to_tensor(occB, dtype=tf.float64)
occC_t = tf.convert_to_tensor(occC, dtype=tf.float64)

In [10]:
# --- NORMAL ORDER HAMILTONIAN -----------
# Calculate 0b, 1b, 2b pieces 
#
# zero-body piece is scalar
# one-body piece is rank 2 tensor
# two-body piece is rank 4 tensor

# - Calculate 0b tensor
E = tf.Variable(0, dtype=tf.float64)
contr_1b = tf.map_fn(lambda i: H1B_t[i,i], holes, dtype=tf.float64)
contr_2b = tf.map_fn(lambda i: H2B_t[i,:,i,:], holes, dtype=tf.float64)

E = tf.assign_add( E, tf.reduce_sum(contr_1b, 0) )
E = tf.assign_add( E, 0.5*tf.reduce_sum(contr_2b, [0,1,2]) )

# - Calculate 1b tensor
f = tf.Variable(tf.identity(H1B_t), dtype=tf.float64)
contr_2b = tf.map_fn(lambda i: H2B_t[:,i,:,i], holes, dtype=tf.float64)
contr_2b = tf.reduce_sum(contr_2b,0) # sum over holes

f = tf.add(f,contr_2b)

# - Calculate 2b tensor
G = tf.identity(H2B_t)

In [11]:
# --- SET UP WHITE'S GENERATOR -----------

# TODO: try reduce_sum, einsum, tensordot?, other methods

def white(f, G, holes, particles):
  
    # - Calculate 1b generator tensor
    eta1B = tf.identity(f)
    eta1Bph = eta1B[particles[0]:particles[-1]+1,holes[0]:holes[-1]+1]
    fhh = f[holes[0]:holes[-1]+1,holes[0]:holes[-1]+1]
    fpp = f[particles[0]:particles[-1]+1,particles[0]:particles[-1]+1]
    Gphph = G[particles[0]:particles[-1]+1, holes[0]:holes[-1]+1, particles[0]:particles[-1]+1, holes[0]:holes[-1]+1]
    Gphph = tf.reduce_sum(Gphph,[0,1]) # reduce to same rank as f
    
    Deltaph = tf.subtract(fpp, tf.add(fhh,Gphph))
    temp = tf.div_no_nan(eta1Bph,Deltaph)
    
    top_block = tf.zeros((4,4), dtype=tf.float64)
    bottom_block = tf.zeros((4,8), dtype=tf.float64)
    conc_top = tf.concat([top_block, temp],1)
    eta1B = tf.concat([conc_top, bottom_block], 0)
    
    # - Calculate 2b generator tensor
    eta2B = tf.zeros(G.shape)
#     return temp
    return eta1B
#     return (eta1B, eta2B)

In [63]:
# --- SET UP WEGNER'S GENERATOR -----------

def wegner(f, G):
# basis = tf.concat([holes, particles],0)

    fd = tf.matrix_diag(tf.diag_part(f))
    fod = tf.subtract(f,fd)
    Gd = tf.linalg.tensor_diag(tf.linalg.tensor_diag_part(G))
    God = tf.subtract(G,Gd)

    # --- 1B piece
    eta1B = tf.Variable(tf.zeros(f.shape, dtype=tf.float64))

    # Calculate 1B-1B contribution
    fd_fod = tf.tensordot(fd,fod,1)
    fd_fod_T = tf.transpose(fd_fod)
    eta1B = tf.assign_add(eta1B, tf.subtract(fd_fod, fd_fod_T))

    # fod_fd = tf.tensordot(fod,fd,1)
    # eta1B = tf.assign_add(eta1B, tf.subtract(fd_fod, fod_fd))


    # Calculate 1B-2B contribution
    fd_God = tf.tensordot(fd, tf.tensordot(occA_t,God,([0,1],[2,0])),([0,1],[2,0]))
    fod_Gd = tf.tensordot(fod, tf.tensordot(occA_t,Gd,([0,1],[2,0])),([0,1],[2,0]))
    eta1B = tf.assign_add(eta1B, tf.subtract(fd_God, fod_Gd))

    # first get n_a - n_b
    # na_nb = tf.constant([[0,0,0,0,1,1,1,1],
    #                      [0,0,0,0,1,1,1,1],
    #                      [0,0,0,0,1,1,1,1],
    #                      [0,0,0,0,1,1,1,1],
    #                      [-1,-1,-1,-1,0,0,0,0],
    #                      [-1,-1,-1,-1,0,0,0,0],
    #                      [-1,-1,-1,-1,0,0,0,0],
    #                      [-1,-1,-1,-1,0,0,0,0]], dtype=tf.float64)


    # Calculate 2B-2B contribution
    Gd_God = tf.tensordot(Gd, tf.tensordot(occC_t,God,([0,1,2],[0,1,2])),([2,3,1],[0,1,2]))
    Gd_God_T = tf.transpose(Gd_God)
    scaled_sub = tf.scalar_mul(tf.constant(0.5,dtype=tf.float64),tf.subtract(Gd_God,Gd_God_T))
    eta1B = tf.assign_add(eta1B, scaled_sub)

    # God_Gd = tf.tensordot(God, tf.tensordot(occC_t,Gd,([0,1,2],[0,1,2])),([0,1,2],[2,3,1]))
    # scaled_sub = tf.scalar_mul(tf.constant(0.5,dtype=tf.float64),tf.subtract(Gd_God,God_Gd))
    # eta1B = tf.assign_add(eta1B, scaled_sub)

    # --- 2B piece
    eta2B = tf.Variable(tf.zeros(G.shape, dtype=tf.float64))

    # Calculate 1B-2B contribution
    fdGod_fodGd_ij = tf.subtract( tf.tensordot(fd,God,([1],[0])), tf.tensordot(fod,Gd,([1],[0])) )
    fdGod_fodGd_ij_T = tf.transpose(fdGod_fodGd_ij, perm=[1,0,2,3])
    ij_term = tf.subtract(fdGod_fodGd_ij,fdGod_fodGd_ij_T)

    # Godfd_Gdfod_ij = tf.subtract( tf.tensordot(God,fd,([0],[1])), tf.tensordot(Gd,fod,([0],[1])) )
    # ij_term = tf.subtract(fdGod_fodGd_ij,Godfd_Gdfod_ij)

    fdGod_fodGd_kl = tf.subtract( tf.tensordot(fd,God,([0],[2])), tf.tensordot(fod,Gd,([0],[2])) )
    fdGod_fodGd_kl_T = tf.transpose(fdGod_fodGd_kl,perm=[0,1,3,2])
    kl_term = tf.subtract(fdGod_fodGd_kl,fdGod_fodGd_kl_T)

    # Godfd_Gdfod_kl = tf.subtract( tf.tensordot(God,fd,([2],[0])), tf.tensordot(Gd,fod,([2],[0])) )
    # kl_term = tf.subtract(fdGod_fodGd_kl,Godfd_Gdfod_kl)

    eta2B = tf.assign_add(eta2B, tf.subtract(ij_term,kl_term))

    # Calculate 2B-2B contribution
    GdGod_occB = tf.tensordot( Gd, tf.tensordot(occB_t,God,([0,1],[0,1])), ([2,3],[0,1]) )
    GodGd_occB = tf.tensordot( God, tf.tensordot(occB_t,Gd,([0,1],[0,1])), ([2,3],[0,1]) )
    scaled_sub = tf.scalar_mul(tf.constant(0.5,dtype=tf.float64), tf.subtract(GdGod_occB,GodGd_occB))
    eta2B = tf.assign_add(eta2B, scaled_sub)

    GdGod_occA = tf.tensordot( Gd, tf.tensordot(occA_t,God,([0,1],[2,0])), ([0,2],[2,0]) )
    GdGod_occA_Tij = tf.transpose(GdGod_occA,perm=[1,0,2,3])
    GdGod_occA_Tkl = tf.transpose(GdGod_occA,perm=[0,1,3,2])
    GdGod_occA_Tijkl = tf.transpose(GdGod_occA,perm=[1,0,3,2])
    sub1 = tf.subtract(GdGod_occA,GdGod_occA_Tij)
    sub2 = tf.subtract(sub1,GdGod_occA_Tkl)
    add3 = tf.add(sub2,GdGod_occA_Tijkl)

    eta2B = tf.assign_add(eta2B, add3)

In [6]:
sess = tf.InteractiveSession()

In [62]:
# indices = tf.constant([[4], [5], [6], [7]])
# updates = tf.gather(f,particles) #tf.constant(tf.gather(f,holes))
# shape = tf.constant([8, 8])
# scatter = tf.scatter_nd(indices, updates, shape)

# indices = tf.constant([[0,1,2,3],[0,1,2,3]])
# updates = tf.constant([[1,2,3,4],[1,2,3,4]])
# ref = tf.Variable([[1,2,3,4],[1,2,3,4]])
# x = tf.constant([[1,2,3,4],[1,2,3,4],[1,2,3,4],[1,2,3,4]])
# top_block = tf.zeros((4,4),dtype=tf.int32)
# bottom_block = tf.zeros((4,8),dtype=tf.int32)
# conc_top = tf.concat([top_block, x],1)
# conc_bot = tf.concat([conc_top, bottom_block], 0)

# ones = tf.ones((4,4))
test = tf.constant([1.,1.,1.,1.])
sess.run(tf.global_variables_initializer())

print(eta2B.eval())
# print(na_nb_t.eval())

# print(x.eval())
# print(y.eval())
# print(conc_bot.eval())

# print(0.5*(tf.subtract(ones,tf.transpose(ones))).eval())
# print(f.shape)
# print(fd.eval())

# print(tf.tensordot(fd,fod,1).eval())
# print(Gd.shape)
# print(tf.tensordot(fd,God,([0,1],[2,0])).eval().shape)
# print(Gd.eval())

# print(white(f,G,holes,particles).eval())
# eta1B,eta2B = white(f,G,holes,particles)
# print(eta1B.eval())
# print(tf.einsum('ijkl->ik',G).eval())
# print(  tf.reduce_sum(tf.map_fn(lambda p: H2B_t[p,:,p,:], particles, dtype=tf.float64),0).eval())
# print(E.eval())
# print(f.eval())
# print(tf.gather(f,particles).eval())
# print(scatter.eval())

[[[[ 0.      0.      0.     ...  0.      0.      0.    ]
   [ 0.      0.      0.     ...  0.      0.      0.    ]
   [ 0.      0.      0.     ...  0.      0.      0.    ]
   ...
   [ 0.      0.      0.     ...  0.      0.      0.    ]
   [ 0.      0.      0.     ...  0.      0.      0.    ]
   [ 0.      0.      0.     ...  0.      0.      0.    ]]

  [[ 0.      0.125   0.     ...  0.      0.      0.    ]
   [-0.125   0.      0.     ...  0.      0.      0.    ]
   [ 0.      0.      0.     ...  0.      0.      0.    ]
   ...
   [ 0.      0.      0.     ...  0.      0.      0.    ]
   [ 0.      0.      0.     ...  0.      0.      0.0625]
   [ 0.      0.      0.     ...  0.     -0.0625  0.    ]]

  [[ 0.      0.      0.     ...  0.      0.      0.    ]
   [ 0.      0.      0.     ...  0.      0.      0.    ]
   [ 0.      0.      0.     ...  0.      0.      0.    ]
   ...
   [ 0.      0.      0.     ...  0.      0.      0.    ]
   [ 0.      0.      0.     ...  0.      0.      0.    ]
   [ 0

In [0]:
sess.close()